# Convolutional Neural Network

### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

import datasets

In [3]:
#mount google drive to this notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load the Training set

In [21]:
training_set = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/Udemy/Deep Learning A-Z/Datasets/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/training_set', # path to training data
                                                           image_size=(64,64), # resize all images to 128x128
                                                           batch_size=32)  # number of images per batch

# Store class_indices before caching and prefetching
class_names = training_set.class_names

Found 8060 files belonging to 2 classes.


### Load the validation set

In [5]:
# here we want to feature scaling this test
validation_set = tf.keras.utils.image_dataset_from_directory('/content/drive/MyDrive/Udemy/Deep Learning A-Z/Datasets/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set', # path to training data
                                                                    image_size=(64,64), # resize all images
                                                                    batch_size=32)  # number of images per batch

Found 2120 files belonging to 2 classes.


prefetching + caching

In [6]:
AUTOTUNE = tf.data.AUTOTUNE
training_set = training_set.cache().prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

# Data Augmentation

In [7]:
#This creates different variations of each image during training → helps the CNN generalize.
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
])

## Part 2 - Building the CNN

### Initialising the CNN

In [8]:
cnn = tf.keras.models.Sequential()

In [9]:
cnn.add(data_augmentation)
cnn.add(layers.Rescaling(1./255))

### Step 1 - Convolution

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [11]:
cnn.maxpool = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)

### Adding a second convolutional layer

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [14]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [15]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

###***add dropout to prevent overfitting

In [16]:
cnn.add(tf.keras.layers.Dropout(0.2))

### Step 5 - Output Layer

In [17]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [18]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [19]:
cnn.fit(x = training_set, validation_data = validation_set, epochs = 25)

Epoch 1/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 2127s 8s/step - accuracy: 0.5452 - loss: 0.6922 - val_accuracy: 0.6509 - val_loss: 0.6283
Epoch 2/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 110s 213ms/step - accuracy: 0.6427 - loss: 0.6389 - val_accuracy: 0.6467 - val_loss: 0.6290
Epoch 3/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 82s 214ms/step - accuracy: 0.6475 - loss: 0.6302 - val_accuracy: 0.7000 - val_loss: 0.5958
Epoch 4/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 55s 217ms/step - accuracy: 0.6759 - loss: 0.6017 - val_accuracy: 0.6736 - val_loss: 0.5968
Epoch 5/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 213ms/step - accuracy: 0.6879 - loss: 0.5870 - val_accuracy: 0.6958 - val_loss: 0.5746
Epoch 6/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 53s 210ms/step - accuracy: 0.7032 - loss: 0.5711 - val_accuracy: 0.6910 - val_loss: 0.5675
Epoch 7/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 82s 210ms/step - accuracy: 0.7123 - loss: 0.5539 - val_accuracy: 0.7193 - val_loss: 0.5486
Epoch 8/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - accuracy: 0.7183 - loss: 0

## Part 4 - Making a single prediction

### import single prediction dataset

In [22]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image_A = tf.keras.utils.load_img('/content/drive/MyDrive/Udemy/Deep Learning A-Z/Datasets/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image_A = tf.keras.utils.img_to_array(test_image_A)
test_image_A = np.expand_dims(test_image_A, axis = 0)
resultA = cnn.predict(test_image_A)
# Use the stored class_names instead of accessing training_set.class_indices
if resultA[0][0] == 1:
  prediction_A = class_names[1] # Assuming class_names[1] is 'dog' and class_names[0] is 'cat' based on directory loading
else:
  prediction_A = class_names[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [23]:
print(prediction_A)

cats


In [24]:
test_image_B = tf.keras.utils.load_img('/content/drive/MyDrive/Udemy/Deep Learning A-Z/Datasets/Deep Learning A-Z/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image_B = tf.keras.utils.img_to_array(test_image_B)
test_image_B = np.expand_dims(test_image_B, axis = 0)
resultB = cnn.predict(test_image_B)
# Use the stored class_names instead of accessing training_set.class_indices
if resultB[0][0] == 1:
  prediction_B = class_names[1] # Assuming class_names[1] is 'dog' and class_names[0] is 'cat' based on directory loading
else:
  prediction_B = class_names[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [25]:
print(prediction_B)

cats
